# Buffer System

In [ ]:
class Buffer
  def initialize(max_items, max_flow)
    @ready_pool = 0
    @downstream_buffer = 0

    @max_items = max_items
    @max_flow = max_flow
  end

  def work(items)
    items = [0, items.round].max
    items = discard_excess_items items
    @ready_pool += items

    @downstream_buffer += release_to_buffer

    @downstream_buffer -= work_on_items
    
    @downstream_buffer
  end
  
  private
  
  def discard_excess_items(items)
    [items, @max_items].min
  end
  
  def release_to_buffer
    t = (rand * @ready_pool).round
    @ready_pool -= t
    t
  end
  
  def work_on_items
    r = (rand * @max_flow).round
    [r, @downstream_buffer].min
  end
end

## Hope for the best

Openloop setup described on page 5.

Average consumption rate = inflow 

In [ ]:
def simulate(consumption_rate = 1.0)
  buffer = Buffer.new 200, 10 * consumption_rate
  data = 1.upto(5000).map do |t|
    units_into_buffer = 5.0 # mean(uniform(0,10))
    queue_length = buffer.work units_into_buffer
    [t, units_into_buffer, queue_length]
  end.transpose

  require 'gnuplot'

  Gnuplot::Plot.new do |plot|
    plot.data << Gnuplot::DataSet.new(data.values_at 0, 2) do |ds|
      ds.title = 'Queue Length'
      ds.with = 'lines'
    end
  end
end

simulate

Average consumption_rate = 0.98 of inflow (slightly below inflow)

In [ ]:
simulate(0.98)

In [ ]:
def target_outflow(t)
  if t < 100
    0.0
  elsif t < 300
    50.0
  else
    10.0
  end
end

class Controller
  def initialize(proportional_gain = 0.0, integral_gain = 0.0)
    @proportional_gain = proportional_gain
    @integral_gain = integral_gain
    @cumulative_error = 0.0
  end
  
  def work(error)
    @cumulative_error += error
    
    @proportional_gain * error# + @integral_gain * @cumulative_errror
  end
end

buffer = Buffer.new 10, 10

def closed_loop(controller, buffer)
  current_outflow = 0.0
  data = 1000.times.map do |t|
    r = target_outflow(t)
    e = r - current_outflow

    inflow = controller.work(e)
    current_outflow = buffer.work(inflow)
    [t, target_outflow(t), e, inflow, current_outflow]
  end.transpose

  Gnuplot::Plot.new do |plot|
    plot.data << Gnuplot::DataSet.new(data.values_at 0, 4) do |ds|
      ds.title = 'Queue Length'
      ds.with = 'lines'
    end

    plot.data << Gnuplot::DataSet.new(data.values_at 0, 1) do |ds|
      ds.title = 'Target Length'
      ds.with = 'lines'
    end
  end
end

$K_p = 0.5$

In [ ]:
controller = Controller.new(0.5)

closed_loop(controller, buffer)

$K_p = 2.0$

In [ ]:
controller = Controller.new(2.0)

closed_loop(controller, buffer)

## Integral Control

$K_p = 1.25$

$K_i = 0.01$

In [ ]:
controller = Controller.new 1.25, 0.01

closed_loop(controller, buffer)